# New York City Guide

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframev
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [3]:
NY_data = "newyork_data.json"

with open(NY_data) as ny_json:
    NY_json = json.load(ny_json)

NY_borough_array = []
NY_neighbor_array = []
NY_neighbor_array_index = []
NY_lat = []
NY_lon = []

for i in range(len(NY_json["features"])):
    NY_borough_array.append(NY_json["features"][i]["properties"]["borough"])
    NY_neighbor_array.append(NY_json["features"][i]["properties"]["name"])
    NY_neighbor_array_index.append(i)
    NY_lat.append(NY_json["features"][i]["geometry"]["coordinates"][1])
    NY_lon.append(NY_json["features"][i]["geometry"]["coordinates"][0])


NY_df = pd.DataFrame(index=NY_neighbor_array_index,columns=["Borough","Neighborhood","Latitude","Longitude"])

for i in range(len(NY_json["features"])):
    NY_df.loc[i] = pd.Series({
        "Borough": NY_borough_array[i],
        "Neighborhood": NY_neighbor_array[i],
        "Latitude": NY_lat[i],
        "Longitude": NY_lon[i]
    }
    )

NY_df.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.8947,-73.8472
1,Bronx,Co-op City,40.8743,-73.8299
2,Bronx,Eastchester,40.8876,-73.8278
3,Bronx,Fieldston,40.8954,-73.9056
4,Bronx,Riverdale,40.8908,-73.9126
5,Bronx,Kingsbridge,40.8817,-73.9028
6,Manhattan,Marble Hill,40.8766,-73.9107
7,Bronx,Woodlawn,40.8983,-73.8673
8,Bronx,Norwood,40.8772,-73.8794
9,Bronx,Williamsbridge,40.881,-73.8574


In [4]:
from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    try:
        geolocator = Nominatim()
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

NY_address = "New York CIty, NY"
NY_geolocator = Nominatim()
NY_location = do_geocode(NY_address)
# NY_latitude = NY_location.latitude
# NY_longitude = NY_location.longitude

NY_latitude = 40.7308619
NY_longitude = -73.9871558

In [5]:
NY_map = folium.Map([NY_latitude,NY_longitude], zoom_start = 10.5)
for lat, lng, borough, neighborhood in zip(NY_df["Latitude"] ,NY_df["Longitude"],
                                          NY_df["Borough"], NY_df["Neighborhood"]):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7
    ).add_to(NY_map)
    

NY_map

In [7]:
CLIENT_ID = 'ZZP3V4PRVFZBON3XKPHJWVSMZUU1V2N1DHUO3ZQCZT0ZYXTO'
CLIENT_SECRET = '5A2O11PRM0ETGTBMBW0SN2V4GSEO0GRC44IQPMMIERCPYOAX'
VERSION = '20180605'

LIMIT = 100
radius = 500

In [8]:
NY_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    NY_latitude,
    NY_longitude,
    radius,
    LIMIT)

In [9]:
NY_url_results = requests.get(NY_url).json()

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
NY_venues = NY_url_results["response"]["groups"][0]["items"]
NY_nearby_venues = json_normalize(NY_venues)
NY_filtered_columns = ["venue.name","venue.categories","venue.location.lat","venue.location.lng"]
NY_nearby_venues = NY_nearby_venues.loc[:,NY_filtered_columns]
NY_nearby_venues["venue.categories"] = NY_nearby_venues.apply(get_category_type,axis=1)
NY_nearby_venues.columns = [col.split(".")[-1] for col in NY_nearby_venues.columns]

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
NY_venues = getNearbyVenues(names=NY_df["Neighborhood"],
                           latitudes = NY_df["Latitude"],
                           longitudes = NY_df["Longitude"])

In [15]:
NY_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
5,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
6,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place
7,Wakefield,40.894705,-73.847201,Baychester Avenue Food Truck,40.892293,-73.843230,Food Truck
8,Wakefield,40.894705,-73.847201,Koss Quick Wash,40.891147,-73.850230,Laundromat
9,Co-op City,40.874294,-73.829939,Dollar Tree,40.870125,-73.828989,Discount Store


In [16]:
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]
NY_onehot.head(10)


,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoo

In [17]:
NY_grouped = NY_onehot.groupby("Neighborhood").mean().reset_index()
NY_grouped.head(10)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoo

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NY_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
NY_neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    NY_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

NY_neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Pharmacy,Spa,Supermarket,Fried Chicken Joint,Donut Shop,Grocery Store,Fast Food Restaurant
1,Annadale,Pizza Place,Restaurant,Sports Bar,Train Station,Food,Diner,Pharmacy,Park,Fast Food Restaurant,Event Space
2,Arden Heights,Pharmacy,Pizza Place,Coffee Shop,Bus Stop,Women's Store,Fish & Chips Shop,Eye Doctor,Factory,Falafel Restaurant,Farm
3,Arlington,Grocery Store,Bus Stop,Deli / Bodega,American Restaurant,Food,Boat or Ferry,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
4,Arrochar,Deli / Bodega,Italian Restaurant,Pizza Place,Bus Stop,Middle Eastern Restaurant,Bagel Shop,Athletics & Sports,Sandwich Place,Hotel,Outdoors & Recreation
5,Arverne,Surf Spot,Metro Station,Bus Stop,Coffee Shop,Sandwich Place,Donut Shop,Board Shop,Playground,Bed & Breakfast,Beach
6,Astoria,Middle Eastern Restaurant,Bar,Hookah Bar,Greek Restaurant,Seafood Restaurant,Bakery,Italian Restaurant,Coffee Shop,Salon / Barbershop,Latin American Restaurant
7,Astoria Heights,Italian Restaurant,Music Venue,Shopping Mall,Bakery,Pizza Place,Hostel,Playground,Bowling Alley,Plaza,Bus Station
8,Auburndale,Italian Restaurant,Athletics & Sports,Furniture / Home Store,Supermarket,Fast Food Restaurant,Mobile Phone Shop,Miscellaneous Shop,Gymnastics Gym,Pet Store,Pharmacy
9,Bath Beach,Pharmacy,Kids Store,Bubble Tea Shop,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Gas Station,Pakistani Restaurant


In [20]:
kclusters = 5

NY_grouped_clustering = NY_grouped.drop("Neighborhood", 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(NY_grouped_clustering)

In [21]:
NY_merged = NY_df

from random import randint
randomArray = []
for i in range(4):
    randomArray.append(randint(0,4))

newLabels = np.append(kmeans.labels_, randomArray)
NY_merged['Cluster Labels'] = newLabels

NY_merged = NY_merged.join(NY_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NY_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.8947,-73.8472,2,Pharmacy,Sandwich Place,Gas Station,Food Truck,Caribbean Restaurant,Dessert Shop,Donut Shop,Ice Cream Shop,Laundromat,Fast Food Restaurant
1,Bronx,Co-op City,40.8743,-73.8299,2,Baseball Field,Restaurant,Bus Station,Ice Cream Shop,Basketball Court,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Park,Grocery Store
2,Bronx,Eastchester,40.8876,-73.8278,0,Caribbean Restaurant,Bus Station,Deli / Bodega,Diner,Bus Stop,Metro Station,Food & Drink Shop,Fast Food Restaurant,Automotive Shop,Seafood Restaurant
3,Bronx,Fieldston,40.8954,-73.9056,0,River,High School,Bus Station,Playground,Plaza,Filipino Restaurant,Exhibit,Eye Doctor,Factory,Falafel Restaurant
4,Bronx,Riverdale,40.8908,-73.9126,2,Park,Bus Station,Gym,Bank,Plaza,Food Truck,Home Service,Fast Food Restaurant,Filipino Restaurant,Field
5,Bronx,Kingsbridge,40.8817,-73.9028,2,Pizza Place,Sandwich Place,Discount Store,Bar,Latin American Restaurant,Deli / Bodega,Mexican Restaurant,Supermarket,Fried Chicken Joint,Donut Shop
6,Manhattan,Marble Hill,40.8766,-73.9107,2,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Donut Shop,Spa,Steakhouse,Supplement Shop,Gym,Tennis Stadium
7,Bronx,Woodlawn,40.8983,-73.8673,2,Pub,Playground,Deli / Bodega,Pizza Place,Food & Drink Shop,Italian Restaurant,Supermarket,Park,Bar,Liquor Store
8,Bronx,Norwood,40.8772,-73.8794,2,Pizza Place,Deli / Bodega,Park,Bank,Pharmacy,Sandwich Place,Athletics & Sports,Mobile Phone Shop,Coffee Shop,Caribbean Restaurant
9,Bronx,Williamsbridge,40.881,-73.8574,2,Bar,Nightclub,Caribbean Restaurant,Soup Place,Women's Store,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [22]:
NY_map_clusters = folium.Map(location = [NY_latitude,NY_longitude], zoom_start = 10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []

for lat ,lng, poi, cluster in zip(NY_df["Latitude"], NY_df["Longitude"],
                                 NY_df["Neighborhood"],NY_df["Cluster Labels"]):
    label = folium.Popup(str(poi)+" Cluster "+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(NY_map_clusters)

NY_map_clusters